In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo
import re
import pandas as pd
import numpy as np
import time

In [4]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

#Nasa News URL Setup
nasa_url = "https://mars.nasa.gov/news/"
browser.visit(nasa_url)
time.sleep(2)

#html and soup
news_html = browser.html
news_soup = BeautifulSoup(news_html, 'html.parser')


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\clair\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# Getting NASA Mars News

In [5]:
#Getting NASA Mars News
title_head = news_soup.find_all('div', class_ = 'content_title')
title_head = title_head[1].text.strip()
print(f'Lastest Title: {title_head}')

Lastest Title: NASA Ingenuity Mars Helicopter Prepares for First Flight


In [9]:
para_head = news_soup.find_all('div', class_ = 'article_teaser_body')
para_head = para_head[0].text.strip()
print(f'Content: {para_head}')

Content: Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.


In [7]:
#browser.quit()

# Getting Space Images

In [8]:
#Setting Image URL
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)

In [25]:
target_img = 'a[class="group  cursor-pointer block"]'
browser.find_by_tag(target_img).click()
image_html = browser.html
image_soup = BeautifulSoup(image_html, 'html.parser')

image_href = image_soup.find_all('a', class_ = 'BaseButton text-contrast-none w-full mb-5 -primary -compact inline-block')
featured_image_url = image_href[0]['href']
featured_image_url

'https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA24551.jpg'

# Mars Facts

In [26]:
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)

In [28]:
facts = pd.read_html(facts_url)
facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [35]:
facts_df = facts[0]
facts_df.columns = ['Fact', 'Value']
facts_df

,Fact,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers
